In [0]:
# -*- coding: utf-8 -*-
#
# オートエンコーダー(cifar）
#

In [0]:
# chainer のインストール
!curl https://colab.chainer.org/install | sh -

# 必要なライブラリイのインストール
!pip install h5py
!pip install pillow
!pip install matplotlib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1379  100  1379    0     0   9317      0 --:--:-- --:--:-- --:--:--  9255
+ apt -y -q install cuda-libraries-dev-9-2
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-9-2 is already the newest version (9.2.148-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
+ pip install -q cupy-cuda92  chainer 
+ set +ex
Installation succeeded!


In [0]:
# Google drive のマウント（認証が必要）
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# データの展開
!unzip -q  '/content/gdrive/My Drive/data/cifar-10.zip'
!ls

cifar-10  gdrive  sample_data


In [0]:
import sys
import numpy as np
import chainer
from chainer import cuda
from chainer import Function
from chainer import report
from chainer import training
from chainer import utils
from chainer import Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList, cuda
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from PIL import Image
import matplotlib.pyplot as plt

In [0]:
# chainerのバージョンの確認
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)
chainer.print_runtime_info()

GPU availability: True
cuDNN availablility: True
Platform: Linux-4.14.65+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 5.0.0
NumPy: 1.14.6
CuPy:
  CuPy Version          : 5.0.0
  CUDA Root             : /usr/local/cuda
  CUDA Build Version    : 9020
  CUDA Driver Version   : 9020
  CUDA Runtime Version  : 9020
  cuDNN Build Version   : 7201
  cuDNN Version         : 7201
  NCCL Build Version    : 2213
iDeep: 2.0.0.post3


In [0]:
# CPU，GPUの確認
!cat /proc/cpuinfo
!cat /proc/driver/nvidia/gpus/0000:00:04.0/information

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
mode

In [0]:
# クラス数
class_num = 10

# 画像の大きさ
XSIZE = 32
YSIZE = 32

# 学習データ数
train_num = 200

# データ
data_vec = np.zeros((class_num,train_num,3,YSIZE,XSIZE), dtype=np.float32)

# 学習のパラメータ
batchsize=10
n_epoch=100
n_train=class_num * train_num

In [0]:
def normalize_img(x):
    # -1 以上を1，0以下を0とする
    return np.float32(0 if x<0 else (1 if x>1 else x))
n_img=np.vectorize(normalize_img)

In [0]:
# データの読み込み
def Read_data( flag ):

    dir = [ "train" , "test" ]
    dir1 = [ "airplane" , "automobile" , "bird" , "cat" , "deer" , "dog" , "frog" , "horse" , "ship" , "truck" ]
    for i in range(class_num):
        print( i )
        for j in range(0,train_num):
            # RGB画像で読み込み
            train_file = "cifar-10/" + dir[ flag ] + "/" + dir1[i] + "/" + str(j) + ".png"
            work_img = Image.open(train_file).convert('RGB')

            # numpyに変換
            temp = np.asarray(work_img).astype(np.float32)

            # (32,32,3)→(3,32,32)に変換
            temp = np.transpose(temp, (2,0,1))
            
            # 入力値の正規化
            data_vec[i][j] = temp / 255.0
    print( " ---- " )

In [0]:
# CNN
class CNN(chainer.Chain):
    # 畳み込みネットワークの設定
    def __init__(self):
        super(CNN, self).__init__(
            # 畳み込み層の設定
            conv1 = L.Convolution2D(3, 128, 3, stride=1, pad=1),
            conv2 = L.Convolution2D(128, 128, 3, stride=1, pad=1),
            conv3 = L.Convolution2D(128, 128, 3, stride=1, pad=1),
            conv4 = L.Convolution2D(128, 128, 3, stride=1, pad=1),
            conv5 = L.Convolution2D(128, 128, 3, stride=1, pad=1),
            conv6 = L.Convolution2D(128, 128, 3, stride=1, pad=1),
            conv7 = L.Convolution2D(128, 3, 3, stride=1, pad=1),
            
            # バッチノーマライゼーション
            bn1 = L.BatchNormalization(128),
            bn2 = L.BatchNormalization(128),
            bn3 = L.BatchNormalization(128),
            bn4 = L.BatchNormalization(128),
            bn5 = L.BatchNormalization(128),
            bn6 = L.BatchNormalization(128),
            bn7 = L.BatchNormalization(3),
        )

    # 損失関数
    def __call__(self, x, y):
        # 誤差二乗和
        return F.mean_squared_error(self.fwd(x), y)

    # 畳み込みネットワーク
    def fwd(self, x):
        # 畳み込み
        # 大きさ 1
        h = F.relu(self.bn1(self.conv1(x)))
        h = F.max_pooling_2d(h, ksize=2,stride=2,pad=0)
        
        # 大きさ 1/2
        h = F.relu(self.bn2(self.conv2(h)))
        h = F.max_pooling_2d(h, ksize=2,stride=2,pad=0)
        
        # 大きさ 1/4
        h = F.relu(self.bn3(self.conv3(h)))
        h = F.max_pooling_2d(h, ksize=2,stride=2,pad=0)

        # 大きさ 1/8
        h = F.relu(self.bn4(self.conv4(h)))
        
        # 逆畳み込み
        h = F.unpooling_2d(h ,2, cover_all=False)

        # 大きさ 1/4
        h = F.relu(self.bn5(self.conv5(h)))
        h = F.unpooling_2d(h ,2, cover_all=False)
        
        # 大きさ 1/2
        h = F.relu(self.bn6(self.conv6(h)))
        h = F.unpooling_2d(h ,2, cover_all=False)
        
        # 大きさ 1
        h = F.relu(self.conv7(h))
        return h

In [0]:
# 学習
def Train():
    # Adamによる更新
    optimizer = optimizers.Adam(alpha=0.0002, beta1=0.5)
    optimizer.setup(model)
    # 正則化
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.00001))
    
    for epoch in range(n_epoch):
        error = 0.0
        for i in range(0, n_train, batchsize):
            x = np.zeros((batchsize, 3, YSIZE, XSIZE), dtype=np.float32)
            y = np.zeros((batchsize, 3, YSIZE, XSIZE), dtype=np.float32)

            # バッチの作成
            for j in range(batchsize):
                rnd_c = np.random.randint(class_num)
                rnd = np.random.randint(train_num)
                x[j,:,:,:] = data_vec[rnd_c][rnd]
                y[j,:,:,:] = data_vec[rnd_c][rnd]
                
            # 入力データ
            xt = Variable(cuda.to_gpu(x))
            yt = Variable(cuda.to_gpu(y))

            # 勾配の初期化→伝播，誤差の計算→逆伝播→パラメータの更新
            model.zerograds()
            loss = model( xt , yt )
            error += loss.data
            loss.backward()
            optimizer.update()
        print( str( epoch ) +  ' : ' + str( error ) )

    # パラメータの保存
    serializers.save_hdf5("/content/gdrive/My Drive/data/model-AE-cifar.h5", model)

In [0]:
# 予測
def Predict():

    # パラメータのロード
    serializers.load_hdf5("/content/gdrive/My Drive/data/model-AE-cifar.h5", model)
    x = np.zeros((1, 3, YSIZE, XSIZE), dtype=np.float32)
    y = np.zeros((1, 3, YSIZE, XSIZE), dtype=np.float32)

    for i in range(class_num):
        for j in range(train_num):
            #　入力データ
            x[0,:,:,:] = data_vec[i][j]
            y[0,:,:,:] = data_vec[i][j]

            xt = Variable(cuda.to_gpu(x))
            yt = Variable(cuda.to_gpu(y))

            # 予測
            predict = cuda.to_cpu( ( model.fwd( xt ) ).data )
            
            if j < 1:
                # 画像の描画
                plt.figure()

                # 元画像の表示
                plt.subplot(1,2,1)
                # (3,32,32) -> (32,32,3)に変換
                work = np.transpose( data_vec[i][j] , (1,2,0) )
                plt.imshow(work,vmin=0, vmax=1)
                plt.title( "Original Image" )
                
                # 復元画像の表示
                plt.subplot(1,2,2)
                # (1,3,32,32) -> (3,32,32)に変換
                work = np.reshape(predict,(3,YSIZE,XSIZE))
                # (3,32,32) -> (32,32,3)に変換
                work = np.transpose( work , (1,2,0) )
                work = n_img( work )
                plt.imshow(work,vmin=0, vmax=1)

                # 画像の保存
                plt.title( "Decode Image(" + str(i) + "," + str(j) + ")" )
                file = "/content/gdrive/My Drive/data/result/decode-" + str(i) + "-" + str(j) + "-result.png"
                print( file )
                plt.savefig(file)
                plt.close()

In [0]:
# GPU
xp = cuda.cupy
cuda.get_device(0).use()
model = CNN()
model.to_gpu()


In [0]:
# データの読み込み
flag = 0
Read_data( flag )

# 学習
Train()

0
1
2
3
4
5
6
7
8
9
 ---- 
0 : 9.61518
1 : 4.666318
2 : 4.1877627
3 : 3.7207274
4 : 3.4896035
5 : 3.2333415
6 : 3.253024
7 : 3.1240504
8 : 2.9420292
9 : 2.7932193
10 : 2.780607
11 : 2.6842515
12 : 2.5781813
13 : 2.5736203
14 : 2.440964
15 : 2.4051588
16 : 2.4214458
17 : 2.209638
18 : 2.248239
19 : 2.2475817
20 : 2.082459
21 : 2.0560465
22 : 2.0832264
23 : 2.0532546
24 : 1.9644841
25 : 1.9455546
26 : 1.8977476
27 : 1.8385531
28 : 1.8306799
29 : 1.793472
30 : 1.8174056
31 : 1.8526107
32 : 1.7661669
33 : 1.7429053
34 : 1.625609
35 : 1.7022465
36 : 1.6413282
37 : 1.6968753
38 : 1.6389155
39 : 1.6046883
40 : 1.6079867
41 : 1.5985799
42 : 1.584913
43 : 1.6179849
44 : 1.5348047
45 : 1.5432298
46 : 1.4556165
47 : 1.4855622
48 : 1.4466555
49 : 1.5786018
50 : 1.5388277
51 : 1.4531841
52 : 1.3910825
53 : 1.3692299
54 : 1.4530121
55 : 1.5398421
56 : 1.3839569
57 : 1.4249223
58 : 1.3759061
59 : 1.3472539
60 : 1.3649538
61 : 1.3118246
62 : 1.4146161
63 : 1.3648837
64 : 1.3327408
65 : 1.3148718
66 : 

In [0]:
flag = 1    

# データの読み込み
Read_data( flag )

# 予測
Predict()

0
1
2
3
4
5
6
7
8
9
 ---- 
/content/gdrive/My Drive/data/result/decode-0-0-result.png
/content/gdrive/My Drive/data/result/decode-1-0-result.png
/content/gdrive/My Drive/data/result/decode-2-0-result.png
/content/gdrive/My Drive/data/result/decode-3-0-result.png
/content/gdrive/My Drive/data/result/decode-4-0-result.png
/content/gdrive/My Drive/data/result/decode-5-0-result.png
/content/gdrive/My Drive/data/result/decode-6-0-result.png
/content/gdrive/My Drive/data/result/decode-7-0-result.png
/content/gdrive/My Drive/data/result/decode-8-0-result.png
/content/gdrive/My Drive/data/result/decode-9-0-result.png


In [0]:
!nvidia-smi

Sat Dec 29 06:08:46 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    72W / 149W |    218MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  